This notebook generates a nice anatomy figure of the gad1b line on top of huc:h2b-mCherry

In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import nrrd
import numpy as np
import tifffile as tiff
from PIL import Image, ImageDraw
from scipy.stats import zscore
from skimage import exposure

In [ ]:
# Loading confocal stacks of onr of the fish
# master = Path(r"\\Funes\Shared\experiments\E0038_confocal\v004_ipn_reference\211108_h2b-mcherry_gad1b-gfp\f2")
master = Path(
    "/Volumes/Shared/experiments/E0038_confocal/v004_ipn_reference/211108_h2b-mcherry_gad1b-gfp/f2"
)
anatomy_h2b = tiff.imread(list(master.glob("*C1-Experiment*"))[0])
anatomy_gad1b = tiff.imread(list(master.glob("*C2-Experiment*"))[0])

### Creating an unprocessed figure of the stacks

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(
    1, 2, figsize=(6, 5), gridspec_kw={"width_ratios": [4, 1]}
)
fig_anatomy.tight_layout()


tmp_view = np.nanmean(anatomy_h2b, axis=0)
ax_anatomy[0].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 100, 0, 100])
ax_anatomy[0].axis("off")

tmp_view = np.nanmean(anatomy_gad1b, axis=0)
ax_anatomy[0].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=20, extent=[0, 100, 0, 100]
)

tmp_view = np.nanmean(anatomy_h2b, axis=2).T
ax_anatomy[1].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 25, 0, 100])
ax_anatomy[1].axis("off")

tmp_view = np.nanmean(anatomy_gad1b, axis=2).T
ax_anatomy[1].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=20, extent=[0, 25, 0, 100]
)
fig_anatomy.tight_layout()
plt.show()
file_name = "anatomy_figure.jpg"
fig_anatomy.savefig(str(master / file_name))

### Creating a zscored masked image of the stack

In [ ]:
# creating a mask (masking out the eyes):
num_planes, nx, ny = np.shape(anatomy_gad1b)
img = Image.new("L", [nx, ny], 0)
x1 = 300
y2 = 650
y1 = 400
x2 = 300
ImageDraw.Draw(img).polygon(
    [
        (x1, 0),
        (x1, y1),
        (0, y2),
        (0, 700),
        (x2, 1024),
        (1024 - x2, 1024),
        (1024, 700),
        (1024, y2),
        (1024 - x1, y1),
        (1024 - x1, 0),
    ],
    outline=1,
    fill=1,
)
mask = np.array(img)
print(np.shape(mask))

In [ ]:
gad1b_norm = np.zeros_like(anatomy_gad1b, dtype=float)
for i in range(num_planes):
    tmp_plane = zscore(anatomy_gad1b[i])
    tmp_plane *= mask
    gad1b_norm[i] = tmp_plane

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(
    1, 2, figsize=(6, 5), gridspec_kw={"width_ratios": [4, 1]}
)
fig_anatomy.tight_layout()


tmp_view = np.nanmean(anatomy_h2b, axis=0)
ax_anatomy[0].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 100, 0, 100])
ax_anatomy[0].axis("off")

tmp_view = np.nanmean(gad1b_norm, axis=0)
ax_anatomy[0].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=1.0, extent=[0, 100, 0, 100]
)

tmp_view = np.nanmean(anatomy_h2b, axis=2).T
ax_anatomy[1].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 25, 0, 100])
ax_anatomy[1].axis("off")

tmp_view = np.nanmean(gad1b_norm, axis=2).T
ax_anatomy[1].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=1.0, extent=[0, 25, 0, 100]
)
fig_anatomy.tight_layout()
plt.show()
file_name = "anatomy_figure_zscore_masked.jpg"
fig_anatomy.savefig(str(master / file_name))

### Creating a masked image of the stack (adjusting gamma)

In [ ]:
gad1b_norm2 = np.copy(anatomy_gad1b)
for i in range(num_planes):
    tmp_plane = anatomy_ga1b[i]
    gad1b_norm2[i] = exposure.adjust_gamma(tmp_plane, 1.5)
    gad1b_norm2[i] *= mask

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(
    1, 2, figsize=(6, 5), gridspec_kw={"width_ratios": [4, 1]}
)
fig_anatomy.tight_layout()


tmp_view = np.nanmean(anatomy_h2b, axis=0)
ax_anatomy[0].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 100, 0, 100])
ax_anatomy[0].axis("off")

tmp_view = np.nanmean(gad1b_norm2, axis=0)
ax_anatomy[0].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=2.0, extent=[0, 100, 0, 100]
)

tmp_view = np.nanmean(anatomy_h2b, axis=2).T
ax_anatomy[1].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0, 25, 0, 100])
ax_anatomy[1].axis("off")

tmp_view = np.nanmean(gad1b_norm2, axis=2).T
ax_anatomy[1].imshow(
    tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=2.0, extent=[0, 25, 0, 100]
)
fig_anatomy.tight_layout()
plt.show()
file_name = "anatomy_figure_zscore_masked2.jpg"
fig_anatomy.savefig(str(master / file_name))